## Data Validation with Voluptuous (Schema Definitions)

In this notebook, we'll use [Voluptuous](https://github.com/alecthomas/voluptuous) to define schemas for our data. We can then use schema checking at different points in our cleanup to ensure we meet criteria. We can then use schema validation exceptions to either mark, set aside or remove unclean / invalid data. 

In [ ]:
import logging
import pandas as pd
from datetime import datetime
from voluptuous import Schema, Required, Range, All, ALLOW_EXTRA
from voluptuous.error import MultipleInvalid, Invalid

In [ ]:
logger = logging.getLogger(0)
logger.setLevel(logging.WARNING)

In [ ]:
sales = pd.read_csv('../data/sales_data.csv')

In [ ]:
sales.head()

In [ ]:
sales.dtypes

In [ ]:
sales['timestamp'].map(lambda x: datetime.strptime(x, 
                                                   '%Y-%m-%d %H:%M:%S'))

### Data Quality Check

In [ ]:
sales.head()

In [ ]:
sales.dtypes

## Defining our first schema

In [ ]:
schema = Schema({
    Required('sale_amount'): All(float, 
                                 Range(min=2.50, max=1450.99)),
}, extra=ALLOW_EXTRA)

In [ ]:
error_count = 0
for s_id, sale in sales.T.to_dict().items():
    try:
        schema(sale)
    except MultipleInvalid as e:
        logging.warning('issue with sale: %s (%s) - %s', 
                        s_id, sale['sale_amount'], e)
        error_count += 1

In [ ]:
error_count

In [ ]:
sales.shape

### Questions we might want to answer:
- Do we have an improperly defined schema?
- Are negative values possibly returns or falsely marked? (data entry proceedures)
- Are higher values combined purchases or special sales? (or potentially fraud?)
- What should we do with our schema and our failing data points?

### Adding a custom Validation Case

In [ ]:
def ValidDate(fmt='%Y-%m-%d %H:%M:%S'):
    return lambda v: datetime.strptime(v, fmt)

In [ ]:
schema = Schema({
    Required('timestamp'): All(ValidDate()),
}, extra=ALLOW_EXTRA)

In [ ]:
error_count = 0
for s_id, sale in sales.T.to_dict().items():
    try:
        schema(sale)
    except MultipleInvalid as e:
        logging.warning('issue with sale: %s (%s) - %s', 
                        s_id, sale['timestamp'], e)
        error_count += 1

In [ ]:
error_count

## So we have valid date structures, what about actual valid dates?

In [ ]:
def ValidDate(fmt='%Y-%m-%d %H:%M:%S'):
    def validation_func(v):
        try:
            assert datetime.strptime(v, fmt) <= datetime.now()
        except AssertionError:
            raise Invalid('date is in the future! %s' % v)
    return validation_func

In [ ]:
schema = Schema({
    Required('timestamp'): All(ValidDate()),
}, extra=ALLOW_EXTRA)

In [ ]:
error_count = 0
for s_id, sale in sales.T.to_dict().items():
    try:
        schema(sale)
    except MultipleInvalid as e:
        logging.warning('issue with sale: %s (%s) - %s', 
                        s_id, sale['timestamp'], e)
        error_count += 1

In [ ]:
error_count

## Exercise: what are some possible reasons for future dates? What should we do with the data and schema?